In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('rec').getOrCreate()

In [3]:
from pyspark.ml.recommendation import ALS

In [4]:
from pyspark.ml.evaluation import RegressionEvaluator

In [5]:
data= spark.read.csv('movielens_ratings.csv', inferSchema=True, header=True)

In [6]:
data.show()

+-------+------+------+
|movieId|rating|userId|
+-------+------+------+
|      2|   3.0|     0|
|      3|   1.0|     0|
|      5|   2.0|     0|
|      9|   4.0|     0|
|     11|   1.0|     0|
|     12|   2.0|     0|
|     15|   1.0|     0|
|     17|   1.0|     0|
|     19|   1.0|     0|
|     21|   1.0|     0|
|     23|   1.0|     0|
|     26|   3.0|     0|
|     27|   1.0|     0|
|     28|   1.0|     0|
|     29|   1.0|     0|
|     30|   1.0|     0|
|     31|   1.0|     0|
|     34|   1.0|     0|
|     37|   1.0|     0|
|     41|   2.0|     0|
+-------+------+------+
only showing top 20 rows



In [7]:
data.describe().show()

+-------+------------------+------------------+------------------+
|summary|           movieId|            rating|            userId|
+-------+------------------+------------------+------------------+
|  count|              1501|              1501|              1501|
|   mean| 49.40572951365756|1.7741505662891406|14.383744170552964|
| stddev|28.937034065088994| 1.187276166124803| 8.591040424293272|
|    min|                 0|               1.0|                 0|
|    max|                99|               5.0|                29|
+-------+------------------+------------------+------------------+



In [8]:
training, test  = data.randomSplit([0.8,0.2])

In [9]:
als = ALS(maxIter=5, regParam=0.01, userCol='userId', itemCol='movieId', ratingCol='rating')

In [11]:
model = als.fit(training)

In [12]:
prediction = model.transform(test)

In [13]:
prediction.show()

+-------+------+------+-----------+
|movieId|rating|userId| prediction|
+-------+------+------+-----------+
|     31|   1.0|    26| -1.9464538|
|     31|   1.0|     5|  1.1861738|
|     31|   2.0|    25|  0.8125978|
|     85|   1.0|    26| -2.1661708|
|     85|   1.0|     5|  2.2120323|
|     85|   1.0|     4|  1.6648918|
|     85|   1.0|     2| -1.2182959|
|     65|   1.0|    19|  1.6484962|
|     65|   5.0|    23|    2.93463|
|     53|   3.0|    13|  2.6427157|
|     53|   1.0|     6|  1.3637179|
|     53|   3.0|    14|  4.2953568|
|     78|   1.0|    28| 0.12535608|
|     78|   1.0|    22|  1.5775352|
|     78|   1.0|     1| 0.60800195|
|     34|   1.0|     4|-0.58397675|
|     34|   1.0|    14| 0.97532904|
|     81|   5.0|    28|  3.5952635|
|     81|   1.0|    22|  1.6074239|
|     81|   1.0|    19|  1.0852456|
+-------+------+------+-----------+
only showing top 20 rows



In [16]:
evaluater = RegressionEvaluator(labelCol='rating', predictionCol='prediction',metricName='rmse')

In [18]:
rmse = evaluater.evaluate(prediction)

In [19]:
rmse

1.6717208323163735

In [31]:
singel_user = test.filter(test['userId']==11).select(['movieId', 'userId'])

In [32]:
singel_user.show()

+-------+------+
|movieId|userId|
+-------+------+
|      6|    11|
|     16|    11|
|     18|    11|
|     23|    11|
|     25|    11|
|     38|    11|
|     40|    11|
|     50|    11|
|     88|    11|
|     89|    11|
+-------+------+



In [33]:
recommendations = model.transform(singel_user)

In [34]:
recommendations.orderBy('prediction', ascending =False).show()

+-------+------+-----------+
|movieId|userId| prediction|
+-------+------+-----------+
|     23|    11|   3.204173|
|     50|    11|  3.0509222|
|     38|    11|  2.5524864|
|     18|    11|  2.0447419|
|      6|    11|  1.7746959|
|     88|    11|  1.5245488|
|     89|    11|0.120066315|
|     25|    11| -0.8618014|
|     40|    11| -1.4940364|
|     16|    11| -1.8413173|
+-------+------+-----------+

